## LSTM  - next word in sequence
https://medium.com/towards-data-science/lstm-by-example-using-tensorflow-feb0c1968537

In [1]:

'''
A Recurrent Neural Network (LSTM) implementation example using TensorFlow..
Next word prediction after n_input words learned from text file.
A story is automatically generated if the predicted word is fed back as input.
Author: Rowel Atienza
Project: https://github.com/roatienza/Deep-Learning-Experiments
'''

from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time



In [3]:

start_time = time.time()
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"


# Target log path
logs_path = '/data'
writer = tf.summary.FileWriter(logs_path)

# Text file containing words for training
training_file = './data/belling_the_cat.txt'



In [4]:
def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [content[i].split() for i in range(len(content))]
    content = np.array(content)
    content = np.reshape(content, [-1, ])
    return content

training_data = read_data(training_file)
print("Loaded training data...")

Loaded training data...


In [5]:
# test
'''c = collections.Counter()  
with open(training_file) as f:
    content = f.readlines()
    #print(content)
    content = [x.strip() for x in content]
    content = [content[i].split() for i in range(len(content))]
   # print(content)
    content = np.array(content)
    print(content)
    content = np.reshape(content, [-1, ])
    print( collections.Counter(content).most_common())
'''
#    for letter, count in c.most_common(3):
#    print('%s: %7d',( letter,count))
        
#collections.Counter( hash(tuple(training_data))).most_common()
#collections.Counter(training_data).most_common()
print(training_data)

['long' 'ago' ',' 'the' 'mice' 'had' 'a' 'general' 'council' 'to'
 'consider' 'what' 'measures' 'they' 'could' 'take' 'to' 'outwit' 'their'
 'common' 'enemy' ',' 'the' 'cat' '.' 'some' 'said' 'this' ',' 'and' 'some'
 'said' 'that' 'but' 'at' 'last' 'a' 'young' 'mouse' 'got' 'up' 'and'
 'said' 'he' 'had' 'a' 'proposal' 'to' 'make' ',' 'which' 'he' 'thought'
 'would' 'meet' 'the' 'case' '.' 'you' 'will' 'all' 'agree' ',' 'said' 'he'
 ',' 'that' 'our' 'chief' 'danger' 'consists' 'in' 'the' 'sly' 'and'
 'treacherous' 'manner' 'in' 'which' 'the' 'enemy' 'approaches' 'us' '.'
 'now' ',' 'if' 'we' 'could' 'receive' 'some' 'signal' 'of' 'her'
 'approach' ',' 'we' 'could' 'easily' 'escape' 'from' 'her' '.' 'i'
 'venture' ',' 'therefore' ',' 'to' 'propose' 'that' 'a' 'small' 'bell'
 'be' 'procured' ',' 'and' 'attached' 'by' 'a' 'ribbon' 'round' 'the'
 'neck' 'of' 'the' 'cat' '.' 'by' 'this' 'means' 'we' 'should' 'always'
 'know' 'when' 'she' 'was' 'about' ',' 'and' 'could' 'easily' 'retire'
 'wh

In [6]:
def build_dataset(words):
    count = collections.Counter(words).most_common() # this orders dictionary entries by most_common
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)
print(dictionary)

{',': 0, 'the': 1, '.': 2, 'and': 3, 'to': 4, 'said': 5, 'a': 6, 'could': 7, 'that': 8, 'cat': 9, 'some': 10, 'this': 11, 'mouse': 12, 'he': 13, 'in': 14, 'we': 15, 'is': 16, 'mice': 17, 'had': 18, 'general': 19, 'enemy': 20, 'but': 21, 'at': 22, 'got': 23, 'up': 24, 'proposal': 25, 'which': 26, 'all': 27, 'of': 28, 'her': 29, 'easily': 30, 'propose': 31, 'bell': 32, 'by': 33, 'she': 34, 'was': 35, 'old': 36, 'long': 37, 'ago': 38, 'council': 39, 'consider': 40, 'what': 41, 'measures': 42, 'they': 43, 'take': 44, 'outwit': 45, 'their': 46, 'common': 47, 'last': 48, 'young': 49, 'make': 50, 'thought': 51, 'would': 52, 'meet': 53, 'case': 54, 'you': 55, 'will': 56, 'agree': 57, 'our': 58, 'chief': 59, 'danger': 60, 'consists': 61, 'sly': 62, 'treacherous': 63, 'manner': 64, 'approaches': 65, 'us': 66, 'now': 67, 'if': 68, 'receive': 69, 'signal': 70, 'approach': 71, 'escape': 72, 'from': 73, 'i': 74, 'venture': 75, 'therefore': 76, 'small': 77, 'be': 78, 'procured': 79, 'attached': 80, '

In [7]:
#test - show first few entries in the training_data, converted to numbers
offset =0
n_input =4
symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
symbols_in_keys
symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
symbols_in_keys
# long ago , the  = 37,38,0,1 
# is followed by mice =  17

array([[[37],
        [38],
        [ 0],
        [ 1]]])

In [8]:
# test - mouse =17 sets 1 on the 17th position
symbols_out_onehot = np.zeros([vocab_size], dtype=float)
symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
symbols_out_onehot

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [9]:
# Parameters
learning_rate = 0.001
training_iters = 50000
display_step = 1000
n_input = 3

# number of units in RNN cell
n_hidden = 512

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}


def RNN(x, weights, biases):

    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)

    # 2-layer LSTM, each layer has n_hidden units.
    # Average Accuracy= 95.20% at 50k iter
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

    # 1-layer LSTM with n_hidden units but with lower accuracy.
    # Average Accuracy= 90.60% 50k iter
    # Uncomment line below to test but comment out the 2-layer rnn.MultiRNNCell above
    # rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

In [ ]:

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    print("Run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("Point your web browser to: http://localhost:6006/")
    while True:
        prompt = "%s words: " % n_input
        sentence = input(prompt)
        sentence = sentence.strip()
        words = sentence.split(' ')
        if len(words) != n_input:
            continue
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            for i in range(32):
                keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
                onehot_pred = session.run(pred, feed_dict={x: keys})
                onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
                symbols_in_keys = symbols_in_keys[1:]
                symbols_in_keys.append(onehot_pred_index)
            print(sentence)
        except:
            print("Word not in dictionary")



Iter= 1000, Average Loss= 4.348834, Average Accuracy= 4.30%
['?', 'the', 'mice'] - [looked] vs [looked]
Iter= 2000, Average Loss= 2.878904, Average Accuracy= 19.30%
['very', 'well', ','] - [but] vs [and]
Iter= 3000, Average Loss= 2.392797, Average Accuracy= 33.40%
['until', 'an', 'old'] - [mouse] vs [was]
Iter= 4000, Average Loss= 2.102652, Average Accuracy= 43.40%
['about', ',', 'and'] - [could] vs [attached]
Iter= 5000, Average Loss= 1.962248, Average Accuracy= 46.80%
['the', 'cat', '.'] - [by] vs [some]
Iter= 6000, Average Loss= 1.644237, Average Accuracy= 55.40%
['ribbon', 'round', 'the'] - [neck] vs [neck]
Iter= 7000, Average Loss= 1.465615, Average Accuracy= 63.10%
[',', 'and', 'attached'] - [by] vs [by]
Iter= 8000, Average Loss= 1.390589, Average Accuracy= 62.00%
['therefore', ',', 'to'] - [propose] vs [could]
Iter= 9000, Average Loss= 1.089963, Average Accuracy= 69.60%
['consists', 'in', 'the'] - [sly] vs [the]
Iter= 10000, Average Loss= 1.061532, Average Accuracy= 69.90%
['you

In [ ]:
symbols_in=np.array([18,6,19])    # which is ['had', 'a', 'general']


In [ ]:
 # Launch the graph
with tf.Session() as session:
    session.run(init)
    while True:
        prompt = "%s words: " % n_input
        sentence = input(prompt)
        sentence = sentence.strip()
        words = sentence.split(' ')
        if len(words) != n_input:
            continue
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            for i in range(32):
                keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
                onehot_pred = session.run(pred, feed_dict={x: keys})
                onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
                symbols_in_keys = symbols_in_keys[1:]
                symbols_in_keys.append(onehot_pred_index)
            print(sentence)
        except:
            print("Word not in dictionary")
